In [1]:
import re, os
import shutil
import pickle
# from logerror import TestError
import pandas as pd

root_folder = "/mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1"
root_folder = "/mnt/hgfs/projlogs/SFW/hq1/Performance Test - Performance_Test1"
# MILL, POND, OPP1, OPP2, FOX
# SOPS, HQ1, BAYVIEW, FIELD1
root_folder = "/mnt/hgfs/projlogs/SFW/SOPS"

In [2]:
import re
import os

class TestError():

    def __init__(self, timestamp, userid, cycleid, uj, stepid, errortype, message, test_folder):
        self.cause = ''
        self.time = timestamp
        self.userid = userid
        self.cycleid = cycleid
        self.stepid = stepid
        self.errortype = errortype
        self.message = message
        self.test_folder = test_folder
        self.uj = uj
        
        self.cause = ''
        self.cause_message = ''
        # print('ERRORTYPE', self.errortype)
        if errortype == 'Success Validation Failure':
            # sv = re.search(r'step \((.+?)\) was not.+?written to "(.+?)"', message)
            sv = re.search(r'failed sub validations: (.+?)\) for step ".+?" was not found in the response. The response received has been written to "(.+?)"', message)
            # print(sv, type(sv))
            # print(sv, type(sv), sv.groups())
            self.validation = sv.group(1)
            self.file = sv.group(2)
#             name_parts = self.file.split('__')
#             self.uj = name_parts[0]

        if errortype == 'Response DD Extraction Failure':
            # ddisrc = re.search(r'response DD item (.+?) used in step (.+?)\.', err)
            ddisrc = re.search(r'Error reading value for response DD item (.+?) used in step .+?', message)
            self.dditem = ddisrc.group(1)
            self.ddi_source = 'gr8' #ddisrc.group(2)
        
        
    
    def show(self):
        return [pd.to_datetime(self.time, infer_datetime_format=True), self.userid, self.cycleid, self.uj, self.errortype, self.cause, 0, self.cause_message]
    
    def getsnapshot(self):
        pass
        
    def getfile(self):
        if self.errortype == 'Success Validation Failure':
            # print("\nfile:",self.file,self.validation,self.file,'\n')
            try:
                with open(self.test_folder + '/' + self.file) as f:
                    html = f.read()
            except FileNotFoundError:
                # fix folder        --- drops the run-name-folder Performance Test - Test28_09_14_51\
                ## fix_filename = self.file[self.file.find('user'):]
                ## print(self.file, fix_filename)
                ##fix1 = self.file.replace('\\','/')
                ##fix2 = fix1.replace(' ', '\\ ')
                
                drfn = os.path.join(self.test_folder, 'user'+self.userid , self.file)
                ######print(self.test_folder, '|', self.userid, '|', self.file, '|', drfn)
                with open(drfn, 'rt') as f:
                    html = f.read()

        return html

    def determine_casuse(self):
        if self.errortype == 'Success Validation Failure':
            html = self.getfile()
            if html.count('<label for="username">User Name:</label>')>0:
                self.setcause('logged off', '')
                return 'logged off'

            kick_match = re.search(r'redirect><\!\[CDATA\[https*://.+?/maximo/webclient/login/logout.jsp.*?\]\]></redirect>', html)
            if kick_match is not None:
                self.setcause('Maximo forcefully signed out the user', '')
                return 'signed out'

            if html.count('title="Please wait...">Please wait...</label>')>0:
                self.setcause('Long Op', '')
                return 'Long Op'
            
            if html.count("addLongOpTimeout('dolongopquerycheck()',")>0:
                self.setcause('Long Op', '')
                return 'Long Op'

            if html.count('MessageWarning.png')>0:
                msg = html[html.find('MessageWarning.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self.setcause('Warning Message', msg)
                return 'Warning Message: ' + msg

            if html.count('st_MessageQuestion.png')>0:
                msg = html[html.find('st_MessageQuestion.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self.setcause('Question Message', msg)
                return 'Question Message: ' + msg
            
            if html.count('st_MessageCritical.png')>0:
                msg = html[html.find('st_MessageCritical.png'):]
                msg = '<' + msg[:msg.find('</table>')]
                msg = msg[:msg.find('</component>')]                
                msg = re.sub(r'<[^>]*?>', '', msg)
                msg = msg.replace('\n','').strip()
                self.setcause('Critical Message', msg)
                return 'Critical Message: ' + msg

            if html.count('>0 - 0 of 0')>0:
                self.setcause('operation resulted in a table with zero rows', 'searchterm: ' + self.validation)
                return 'table with zero rows'

            break_index = html.find('---------------Response-----------------')
            html_request_only = html[:break_index]
            html_response_only = html[break_index + 40:]
            if html_response_only.count(self.validation)>0:
                self.setcause('validation bug', 'searchterm: ' + self.validation)
                return 'validation bug (' + self.file + ')'

            ###  --- COB specific errors ---
            if  html_response_only.count('"id":"0_APPRSS_OPTION","text":"APPRSS"') and not html_response_only.count('Approved'):
                self.setcause('missing "Approved" option', '')
                return 'missing "Approved" option'

            if html_request_only.count('targetId%22%3A%22mx387') and html_response_only.count('title="1 - 2 of 2">1 - 2'):
                self.setcause('dynamic response', 'missing reference of WO field, thus cant validate')
                return 'missing reference of WO field, thus cant validate'

            if html_request_only.count('<command>ISWM-RECORDFAILUREREPORT</command>'):
                self.setcause('response lacks confirmation of recordid', 'response lacks confirmation of recordid')
                return 'response lacks confirmation of recordid'            
            ### --- ==================== ---

            self.setcause('unknown validation fail', html)
            return html
        else:
            self.setcause(self.errortype, '')
            return '!!!!'

    def setcause(self, cause, message):
        self.cause = cause
        self.cause_message = message
        
    def set_maxloop(self, length, start_cycle, end_cycle):
        self.loop_len = length
        self.loop_start_cycle = start_cycle
        self.loop_end_cycle = end_cycle
    

In [3]:
test_folder = root_folder #+ '/' + locationa
files = [ f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder,f)) and f.find('.log.')>0 ]
# print('preliminary files:', files)
files = [ f for f in files if f.find('lck')==-1 ]

# print('for ', location, 'found log files: ', files)
print('found', len(files), 'log files: ', files)

found 1 log files:  ['SOPS2UA5101XHD_0.log.1']


In [4]:
# FNAME = "LocalInjectorCOBAPPSTS03_0.log.1"

if len(files)==1:
    FNAME = files[0]
elif len(files)>1:
    print('multiple files - merging into _log.all')
    FNAME = '_log.all'
    with open(os.path.join(test_folder, FNAME), 'wb') as wfd:
        for file in files:
            with open(os.path.join(test_folder, file),'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*100)
                #100MB per writing chunk to avoid reading big file into memory.

else:
    if not FNAME:
        print('no files found - please assign FNAME manually')
        exit(1)
    

In [5]:
print('processing file: ', test_folder + '/' + FNAME)

with open(test_folder + '/' + FNAME,'r') as f:
    alllog = f.read()

errorlog = re.findall(r'(?<=\n).+?\[SEVERE\].+?id \d+?,[\w\s]+?\):.+?(?=\n)', alllog)
print('severe count:', len(errorlog))

processing file:  /mnt/hgfs/projlogs/SFW/SOPS/SOPS2UA5101XHD_0.log.1
severe count: 6938


In [6]:
snap = r'<GENERAL_DATA>\n\s+?<APPLICATION>([^<]+?)</APPLICATION>\n\s+?<USER_JOURNEY>([^<]+?)</USER_JOURNEY>\n\s+?<STEP>([^<]+?)</STEP>\n\s+?<ERROR>([^<]+?)</ERROR>\n\s+?<INJECTOR_NAME>([^<]+?)</INJECTOR_NAME>\n\s+?<START_DATE>([^<]+?)</START_DATE>\n\s+?<RUN_NAME>([^<]+?)</RUN_NAME>\n\s+?<BROWSER_PROFILE>([^<]+?)</BROWSER_PROFILE>\n\s+?<DATE>([^<]+?)</DATE>\n\s+?<PROCESS_NAME>([^<]+?)</PROCESS_NAME>\n\s+?<PROCESS_NUMBER>([^<]+?)</PROCESS_NUMBER>\n\s+?<USER_ID>([^<]+?)</USER_ID>\n\s+?<CYCLE_ID>([^<]+?)</CYCLE_ID>\n\s+?<CURRENT_USERS>([^<]+?)</CURRENT_USERS>\n\s+?<CURRENT_CONNECTIONS>([^<]+?)</CURRENT_CONNECTIONS>\n\s+?</GENERAL_DATA>'

snaplog = re.findall(snap, alllog)
print('snapshots:', len(snaplog))

snapshots: 7983


In [7]:
print(snaplog[0], snaplog[0][-4], snaplog[0][-3])

('Maximo', 'Creating Communication Log', 'Create Communication', 'The validation criteria (validation: Record has been saved.&quot;&amp;&amp;title=&quot;Template&quot;) (failed sub validations: , title=&quot;Template&quot;) for step &quot;Create Communication&quot; was not found in the response. The response received has been written to &quot;Creating_Communication_Log__Create_Communication__0000_0009_2018-01-29_01-44-14.data&quot;', 'SOPS', '29/01/2018 01:42:46.000 PST', 'Performance Test1', 'Internet Explorer 10', '29/01/2018 01:44:14.964 PST', '2UA5101XHD-0', '0', '15', '0', '17', '1') 15 0


In [8]:
# with open(os.path.join(test_folder, 'snaps.dump.txt'), 'wt') as f:
#     f.write('\n'.join(map(str, snaplog)))

In [9]:
print([ z[1] for z in snaplog if z[-4]=='8' and z[-3]=='500'])

['Open and Assign CM WO as Supervisor', 'Open and Assign CM WO as Supervisor']


In [10]:
errors = []
vusers = {}
for err in errorlog:
    time = err[:20]
    usr = re.search(r'user.(\d+?),.cycle', err)
    user = usr.group(1)
    user = str(user).rjust(4,'0')
    cyc = re.search(r'cycle.(\d+?),.step', err)
    cycle = cyc.group(1)
    stp = re.search(r'step\s+?(.+?),\s+?id', err)
    step = stp.group(1)
    errtype = re.search(r'status.(.+?)\)', err)
    error_type = errtype.group(1)
    message = err[err.find('):'):]
    
    try:
        uj = [ z[1] for z in snaplog if z[-4]==str(int(user)) and z[-3]==cycle][0]
    except IndexError:
        uj = ''
        print('failed to find snapshot data for user', user, 'cycle', cycle)
        
    
    new_record = TestError(time, user, cycle, uj, step, error_type, message, test_folder)
    errors.append(new_record)
    if user in vusers.keys():
        vusers[user].append(new_record)
    else:
        vusers[user] = [new_record]

print('errors:', len(errors))
pickle.dump((errors, vusers), open(os.path.join(test_folder, 'errors.pickle'), 'wb' ))

errors: 6938


In [11]:
# errors, vusers = pickle.load(open(os.path.join(test_folder, 'errors.pickle'), 'rb'))

In [33]:
sorted_keys = sorted(vusers.keys())
loops = {}
print(">> user, i, e.time, e.cycleid, e.errortype, etype, ")
print('erroring users', len(sorted_keys))
for user in sorted_keys:
    print('\nuser ' + user + ' err count:', len(vusers[user]))
    prior_cyc_index = -1
    consecutive_count = 0
    consecutivness = ''
    max_consecutive = -1
    max_start = -1
    max_end = -1
    for i in range(len(vusers[user])-1,-1,-1):
        e = vusers[user][i]
        try:
            etype = e.validation
        except AttributeError:
            try:
                etype = e.dditem
            except AttributeError:
                etype = '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

        # print('pre: cyc', e.cycleid, ' prior', prior_cyc_index, ' consecutive_count', consecutive_count, 'consecutivness', consecutivness )
        # determine consecutivness
        if prior_cyc_index == -1:
            prior_cyc_index = int(e.cycleid)
        else:
            if prior_cyc_index - int(e.cycleid) <= 1 and i != 0:
                consecutive_count += 1
                prior_cyc_index = int(e.cycleid)
            else:
                consecutivness = ';' + str(e.cycleid) + '-' + str(int(e.cycleid) + consecutive_count) + consecutivness
                if consecutive_count > max_consecutive:
                    max_consecutive = consecutive_count
                    max_start = e.cycleid
                    max_end = str(int(e.cycleid) + consecutive_count)
                consecutive_count = 0
                prior_cyc_index = int(e.cycleid)
        cause = e.determine_casuse()
    if max_consecutive > 0:
        e.set_maxloop(max_consecutive, max_start, max_end)
        loops[user] = e.cycleid
        print('~~ max error loop:: length', max_consecutive, 'start cycle:', max_start, 'end cycle:', max_end)
    if len(consecutivness):
        print('~~', consecutivness[1:])

print(end='\n')
print('8'*80)
print("e.time, user, e.cycleid, e.errortype, e.cause, usr err count")

pickle.dump((errors, vusers), open(os.path.join(test_folder, 'errors2.pickle'), 'wb' ))

>> user, i, e.time, e.cycleid, e.errortype, etype, 
erroring users 13

user 0000 err count: 1586
~~ max error loop:: length 1583 start cycle: 34 end cycle: 1617
~~ 0-0;34-1617

user 0001 err count: 1568
~~ max error loop:: length 1562 start cycle: 176 end cycle: 1738
~~ 0-0;18-18;21-21;28-28;176-1738

user 0002 err count: 1571
~~ max error loop:: length 1566 start cycle: 163 end cycle: 1729
~~ 0-0;22-22;30-30;163-1729

user 0006 err count: 1

user 0007 err count: 2
~~ 76-76

user 0008 err count: 342
~~ max error loop:: length 320 start cycle: 194 end cycle: 514
~~ 0-0;41-41;43-43;165-165;167-168;173-173;178-179;181-181;183-192;194-514

user 0009 err count: 337
~~ max error loop:: length 320 start cycle: 190 end cycle: 510
~~ 0-0;46-46;88-88;126-126;136-136;150-150;169-170;178-179;183-187;190-510

user 0010 err count: 221
~~ max error loop:: length 219 start cycle: 0 end cycle: 219
~~ 0-219

user 0011 err count: 223
~~ max error loop:: length 221 start cycle: 145 end cycle: 366
~~ 145-3

In [13]:
# errors, vusers = pickle.load(open(os.path.join(test_folder, 'errors2.pickle'), 'rb'))

In [34]:
# list(vusers.keys())[:5]
print(vusers['0008'][0].show(), loops['0008'])

[Timestamp('2018-01-29 01:46:38'), '0008', '0', 'Open and Assign CM WO as Supervisor', '500', '500', 0, ''] 0


In [40]:
causes = {}

accu = []

for user in sorted_keys:
    try:
        print(user)
        print(int(loops[user]))
        e = vusers[user][int(loops[user])]
    except KeyError:
        e = vusers[user][0]
    msg = ''
    #if e.cause in ['Warning Message', 'Critical Message', 'Question Message', 'dynamic response']:
    #    msg = e.cause_message
    error_details = e.show()
    error_details[-2] += len(vusers[user])
    accu.append(error_details)
    
    if e.cause in causes.keys():
        causes[e.cause] += 1
    else:
        causes[e.cause] = 1

print('count of erroring users:', len(sorted_keys))
for k,v in causes.items():
    print('cause', k, 'has count', v)

data = pd.DataFrame(accu, columns=['time', 'user', 'cycle', 'uj', 'err_class', 'err_cause', 'err_count', 'message'])

0000
0
0001
0
0002
0
0006
0007
0008
0
0009
0
0010
0
0011
145
0012
19
0013
12
0014
0
0015
0
count of erroring users: 13
cause 500 has count 5
cause Response DD Extraction Failure has count 1
cause Long Op has count 2
cause Critical Message has count 3
cause icon, id 3231, status Success Validation Failure has count 2


In [41]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999
data

,time,user,cycle,uj,err_class,err_cause,err_count,message
0,2018-01-29 01:46:31,0000,0,Open and Change to COMP as Supervisor,500,500,1586,
1,2018-01-29 01:46:31,0001,0,Open and Change to COMP as Supervisor,500,500,1568,
2,2018-01-29 01:46:32,0002,0,Open and Change to COMP as Supervisor,500,500,1571,
3,2018-01-29 02:59:17,0006,23,Advanced Search of WorkOrder by Asset,Response DD Extraction Failure,Response DD Extraction Failure,1,
4,2018-01-29 06:08:35,0007,76,Advanced Search of WorkOrder by Asset,Success Validation Failure,Long Op,2,
5,2018-01-29 01:46:38,0008,0,Open and Assign CM WO as Supervisor,500,500,342,
6,2018-01-29 01:46:29,0009,0,Open and Assign CM WO as Supervisor,500,500,337,
7,2018-01-29 01:44:20,0010,0,"Open,edit SR and Convert to WO",Success Validation Failure,Long Op,221,
8,2018-01-29 08:07:33,0011,290,"Open,edit SR and Convert to WO",Success Validation Failure,Critical Message,223,"BMXAA7837E - An error occured that prevented the GISCREATEUPDATESRWOPOINT script for the GISCREATEUPDSRPT_OLP launch point from running.\tjava.lang.Exception: java.lang.Exception: Unable to copy Maximo attributes to GIS feature.Traceback &#040;most recent call last&#041;: File: &quot;&lt;script&gt;&quot;, line 1102, in &lt;module&gt;MXSystemException: BMXAA4211E - Database error number 1000 has occurred when operating on SRPOINT : Objectid=404,500. Report the error to the owner of the deployment. in &lt;script&gt; at line number 1105"
9,2018-01-29 04:45:49,0012,58,Open WO finding and FIELDCOMP,"icon, id 3231, status Success Validation Failure","icon, id 3231, status Success Validation Failure",137,
